In [1]:
import numpy as np
from PIL import Image
import random
np.random.seed(12)
random.seed(12)

In [2]:
def int_of_bytes(bytes):
    return int.from_bytes(bytes, "big")


def open_image(path):
    f = open(path, "rb")
    f.read(4)  # magic number
    num_images = int_of_bytes(f.read(4))
    rows = int_of_bytes(f.read(4))
    columns = int_of_bytes(f.read(4))
    f.close()
    data = np.fromfile(path, dtype="uint8")
    data = data[16:]
    return np.reshape(data, (num_images, rows, columns))


def display_image(np_array):
    im = Image.fromarray(np_array)
    im = im.convert("L")
    display(im)


def open_label(path):
    f = open(path, "rb")
    f.read(4)  # magic number
    num_images = int_of_bytes(f.read(4))
    f.close()
    data = np.fromfile(path, dtype="uint8")
    data = data[8:]
    return np.reshape(data, (num_images))


In [3]:
images_train = open_image("train-images-idx3-ubyte")
labels_train = open_label("train-labels-idx1-ubyte")
images_test = open_image("t10k-images-idx3-ubyte")
labels_test = open_label("t10k-labels-idx1-ubyte")

In [4]:
def p_sigmoid(x):
    y = sigmoid(x)
    return y * (1 - y)


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def prepare_data(images, labels):
    return [(i.reshape(i.size, 1), label_to_vector(l)) for i, l in zip(images, labels)]


def label_to_vector(label: int):
    r = np.zeros((10, 1))
    r[label][0] = 1
    return r


def vector_to_label(vec):
    return np.argmax(vec)



In [5]:
class Network:
    def __init__(self, sizes):
        self.sizes = sizes
        self.num_layers = len(sizes)
        self.biases = [np.random.randn(x, 1) for x in sizes[1:]]
        self.weights = [np.random.randn(x, y) for x, y in zip(sizes[1:], sizes)]

    def sdg(self, training_data, epochs, mini_batch_size, eta, test_data):
        n = len(training_data)
        for i in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[x:x + mini_batch_size] for x in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print(f'Epoch {i}: {self.evaluate(test_data)} / {len(test_data)}')
            else:
                print(f'Epoch {i} complete')

    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.biases = [b - (eta / len(mini_batch)) * nb for b, nb in zip(self.biases, nabla_b)]
        self.weights = [w - (eta / len(mini_batch)) * nw for w, nw in zip(self.weights, nabla_w)]

    def backprop(self, x, y):
        zs = []
        acs = [x]
        a = x
        delta_nabla_b = [np.zeros(b.shape) for b in self.biases]
        delta_nabla_w = [np.zeros(w.shape) for w in self.weights]
        z = None
        for w, b in zip(self.weights, self.biases):
            z = np.dot(w, a) + b
            a = sigmoid(z)
            zs.append(z)
            acs.append(a)
        output_error = (acs[-1] - y) * p_sigmoid(z[-1])
        delta_nabla_b[-1] = output_error
        delta_nabla_w[-1] = output_error @ acs[-2].transpose()

        for i in range(2, self.num_layers):
            output_error = (self.weights[-i + 1].transpose() @ output_error) * p_sigmoid(z[-i])
            delta_nabla_b[-i] = output_error
            delta_nabla_w[-i] = output_error @ acs[- i - 1].transpose()
        return delta_nabla_b, delta_nabla_w

    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == vector_to_label(y)) for (x, y) in test_results)

    def feedforward(self, inp):
        for b, w in zip(self.biases, self.weights):
            inp = sigmoid(np.dot(w, inp) + b)
        return inp


In [6]:
network = Network([784, 30, 10])
def main():
    training_data = prepare_data(images_train, labels_train)
    test_data = prepare_data(images_test, labels_test)
    network.sdg(training_data, 30, 20, 3.0, training_data)


In [7]:
main()

/var/folders/k8/c8_zkt093fv9l1wmmxjpjt100000gn/T/ipykernel_41628/363644159.py:7: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch 0: 35743 / 60000
Epoch 1: 34597 / 60000
Epoch 2: 30398 / 60000
Epoch 3: 33484 / 60000
Epoch 4: 36418 / 60000
Epoch 5: 36009 / 60000
Epoch 6: 29607 / 60000
Epoch 7: 30412 / 60000
Epoch 8: 33967 / 60000
Epoch 9: 34610 / 60000
Epoch 10: 34122 / 60000
Epoch 11: 33782 / 60000
Epoch 12: 31691 / 60000
Epoch 13: 31876 / 60000
Epoch 14: 31400 / 60000
Epoch 15: 30104 / 60000
Epoch 16: 30388 / 60000
Epoch 17: 33233 / 60000
Epoch 18: 30304 / 60000
Epoch 19: 31173 / 60000
Epoch 20: 34832 / 60000
Epoch 21: 35949 / 60000
Epoch 22: 37589 / 60000
Epoch 23: 40271 / 60000
Epoch 24: 39571 / 60000
Epoch 25: 40481 / 60000
Epoch 26: 39643 / 60000
Epoch 27: 40950 / 60000
Epoch 28: 39829 / 60000
Epoch 29: 40290 / 60000


In [8]:
def predict_label(image):
    vl = network.feedforward(image.reshape(image.size, 1))
    return vector_to_label(vl)

In [9]:
display_image(images_test[0])